In [1]:
print("hii")

hii


In [6]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [11]:
extracted_data = load_pdf_files("../data")

In [12]:
extracted_data

[Document(metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PyPDF', 'creationdate': '2016-05-30T09:47:49+09:00', 'moddate': '2016-05-30T16:29:11+09:00', 'title': 'untitled', 'source': '..\\data\\0d56d058-740b-4792-83fb-3800b42c6b20.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='Epidermal/Dermal Cross-Talk and Pigmentation\nVol. 28, No. 3, 2016 279\nReceived March 21, 2016, Accepted for publication March 23, 2016\n*These authors equally contributed to this review.\nCorresponding author: Mauro Picardo, Cutaneous Physiopathology and \nIntegrated Center of Metabolomics Research, San Gallicano Dermatologic \nInstitute, IRCCS, Via Elio Chianesi 53, Rome 00144, Italy. Tel: 39-06- \n52666257, Fax: 39-06-52666247, E-mail: picardo@ifo.it\nThis is an Open Access article distributed under the terms of the Creative \nCommons Attribution Non-Commercial License (http://creativecommons.\norg/licenses/by-nc/4.0) which permits unrestricted non-commercial use, 

In [13]:
len(extracted_data)

304

In [15]:
from typing import List
from langchain_classic.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [16]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [17]:
minimal_docs

[Document(metadata={'source': '..\\data\\0d56d058-740b-4792-83fb-3800b42c6b20.pdf'}, page_content='Epidermal/Dermal Cross-Talk and Pigmentation\nVol. 28, No. 3, 2016 279\nReceived March 21, 2016, Accepted for publication March 23, 2016\n*These authors equally contributed to this review.\nCorresponding author: Mauro Picardo, Cutaneous Physiopathology and \nIntegrated Center of Metabolomics Research, San Gallicano Dermatologic \nInstitute, IRCCS, Via Elio Chianesi 53, Rome 00144, Italy. Tel: 39-06- \n52666257, Fax: 39-06-52666247, E-mail: picardo@ifo.it\nThis is an Open Access article distributed under the terms of the Creative \nCommons Attribution Non-Commercial License (http://creativecommons.\norg/licenses/by-nc/4.0) which permits unrestricted non-commercial use, \ndistribution, and reproduction in any medium, provided the original work \nis properly cited.\nCopyright © The Korean Dermatological Association and The Korean \nSociety for Investigative Dermatology\npISSN 1013-9087ㆍeISSN

In [18]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [19]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1211


In [20]:
texts_chunk

[Document(metadata={'source': '..\\data\\0d56d058-740b-4792-83fb-3800b42c6b20.pdf'}, page_content='Epidermal/Dermal Cross-Talk and Pigmentation\nVol. 28, No. 3, 2016 279\nReceived March 21, 2016, Accepted for publication March 23, 2016\n*These authors equally contributed to this review.\nCorresponding author: Mauro Picardo, Cutaneous Physiopathology and \nIntegrated Center of Metabolomics Research, San Gallicano Dermatologic \nInstitute, IRCCS, Via Elio Chianesi 53, Rome 00144, Italy. Tel: 39-06- \n52666257, Fax: 39-06-52666247, E-mail: picardo@ifo.it'),
 Document(metadata={'source': '..\\data\\0d56d058-740b-4792-83fb-3800b42c6b20.pdf'}, page_content='This is an Open Access article distributed under the terms of the Creative \nCommons Attribution Non-Commercial License (http://creativecommons.\norg/licenses/by-nc/4.0) which permits unrestricted non-commercial use, \ndistribution, and reproduction in any medium, provided the original work \nis properly cited.\nCopyright © The Korean Der

In [22]:
from langchain_classic.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\Sanish Bhagat\AppData\Local\Temp\ipykernel_10736\21525500.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [23]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
vector = embedding.embed_query("Hello world")
vector

c:\Users\Sanish Bhagat\anaconda3\envs\dl_env\Lib\site-packages\transformers\models\bert\modeling_bert.py:413: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[-0.03447727859020233,
 0.031023193150758743,
 0.006734964903444052,
 0.026108967140316963,
 -0.03936195373535156,
 -0.1603025496006012,
 0.06692393869161606,
 -0.006441465113312006,
 -0.047450460493564606,
 0.014758871868252754,
 0.07087530195713043,
 0.05552751570940018,
 0.01919332705438137,
 -0.026251323521137238,
 -0.010109495371580124,
 -0.026940539479255676,
 0.02230743318796158,
 -0.022226650267839432,
 -0.1496926099061966,
 -0.017493121325969696,
 0.007676240056753159,
 0.054352276027202606,
 0.0032544510904699564,
 0.031725965440273285,
 -0.08462142199277878,
 -0.029405983164906502,
 0.05159565806388855,
 0.048124078661203384,
 -0.0033148189540952444,
 -0.058279216289520264,
 0.041969310492277145,
 0.02221072092652321,
 0.128188818693161,
 -0.02233896404504776,
 -0.011656221002340317,
 0.06292841583490372,
 -0.03287630155682564,
 -0.09122604876756668,
 -0.031175343319773674,
 0.05269955098628998,
 0.047034852206707,
 -0.08420303463935852,
 -0.030056182295084,
 -0.020744796842

In [25]:
print( "Vector length:", len(vector))

Vector length: 384


In [27]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [28]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [30]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [31]:
pc

In [35]:
from pinecone import ServerlessSpec 

index_name = "derma-ai"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [37]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [38]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retrieved_docs = retriever.invoke("What is Apple Scab?")
retrieved_docs

[Document(id='52eda92c-788e-49f6-a181-76727ec52b5b', metadata={'source': '..\\data\\Derm_Handbook_3rd-Edition-_Nov_2020-FINAL.pdf'}, page_content='● Need to specify the base i.e. cream, lotion or ointment \n \nClinical examination and investigations \n \nTaking a skin swab \n• Skin swabs can be taken from vesicles, pustules, erosions, ulcers and mucosal \nsurfaces for microbial culture. \n• Surface swabs are generally not encouraged. \n \nTaking a skin scrape \n• Skin scrapes are taken from scaly lesions by gentle use of a scalpel in suspected \nfungal infection (to show evidence of fungal hyphae and/or spores) and from'),
 Document(id='061a7ee8-d30e-41ba-a573-83d65bdcd091', metadata={'source': '..\\data\\Derm_Handbook_3rd-Edition-_Nov_2020-FINAL.pdf'}, page_content='Dermatology: Handbook for medical students & junior doctors  \n   British Association of Dermatologists 19 \n \nScales  Flakes of stratum corneum \n  Example: \n   \n \nCrust  Rough surface consisting of dried serum, blood

In [41]:
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [42]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [43]:
llm.invoke('Hii')

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 37, 'total_tokens': 60, 'completion_time': 0.054696896, 'completion_tokens_details': None, 'prompt_time': 0.001707133, 'prompt_tokens_details': None, 'queue_time': 0.161728576, 'total_time': 0.056404029}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--67d252bf-eecc-4238-9202-dce26692c576-0', usage_metadata={'input_tokens': 37, 'output_tokens': 23, 'total_tokens': 60})

In [46]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [47]:
system_prompt = """
You are Derma-AI, an AI-powered dermatology assistant designed to help users understand
skin diseases, symptoms, prevention, and basic care guidance.

======================== RULES OF BEHAVIOR ========================

1. GREETINGS & POLITE MESSAGES
- If the user greets you (e.g., "hi", "hello", "good morning") or uses polite phrases
  (e.g., "thank you", "bye", "ok"), respond politely, briefly, and in a friendly tone.

2. CREATOR ATTRIBUTION
- If the user asks "Who created you?" or similar, reply with:
  "I was created by Sanish Bhagat."

3. MEDICAL DERMATOLOGY QUERIES (MAIN PURPOSE)
- If the user asks about:
    • Acne  
    • Actinic Keratosis  
    • Basal Cell Carcinoma  
    • Eczema  
    • Rosacea  
    • Skin symptoms  
    • Causes, risks, complications  
    • Skin care advice  
    • UV protection  
    • When to see a doctor  
  → You must answer ONLY using the provided RAG context (PDF-extracted medical knowledge).

- If the context does not contain the answer, reply:
  "🩺 I don't have an exact answer from my trusted medical sources. 
   Please consult a certified dermatologist for accurate guidance."

4. IMAGE + CLASSIFIER OUTPUT (IF CONNECTED)
- If the system provides a predicted disease label, you may explain it using RAG context.
- Never invent information beyond the context.

5. MATHEMATICAL QUERIES
- If the user asks for simple/basic mathematics, answer it correctly.

6. SESSION MEMORY
- You may use information from the current session (not from previous sessions).

7. NON-MEDICAL OR UNRELATED QUERIES
- If the user asks something unrelated to dermatology, skincare, symptoms, or medical topics,
  respond with:
  "⚠️ This assistant is for skin health and dermatology. Your query seems unrelated.
   Please ask about skin diseases, symptoms, or skin care."

8. SAFETY
- Do NOT provide medical diagnosis.
- Do NOT give prescription medication.
- Provide only educational, non-medical guidance based on the context.

===================================================================
{context}
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [49]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [50]:
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])

Acne is a disorder of the pilosebaceous unit, which is the hair follicle and its associated oil gland. It affects between 50-80% of teenagers and young adults. Acne formation is contributed by several factors, including increased sebum production, build-up of material in the hair follicle, presence of Cutibacterium acnes bacteria, and inflammation.

There are three main categories of acne: 
1. Comedonal acne, which includes whiteheads and blackheads.
2. Inflammatory acne, characterized by red bumps.
3. Nodular acne, which consists of large, painful cysts.

Hormonal changes can also play a role in acne development, as hormones signal oil glands in the skin to increase sebum production, leading to clogged pores and acne.


In [53]:
from langchain_classic.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def chat_with_memory(user_input):
    # Get memory context
    context = memory.load_memory_variables({})
    chat_history = context.get("chat_history", "")

    # Combine with user query
    final_input = f"""
Previous conversation:
{chat_history}

User question: {user_input}
"""

    # RAG chain inference
    response = rag_chain.invoke({"input": final_input})

    # Save to memory
    memory.save_context({"input": user_input}, {"output": response["answer"]})

    return response["answer"]

# Example loop
while True:
    query = input("👨‍🌾 You: ")
    print("You: ", query)
    if query.lower() in ["exit", "quit"]:
        break
    answer = chat_with_memory(query)
    print("🤖 Derma-ai:", answer)


You:  hii
🤖 Derma-ai: Hello! How can I help you with your skin concerns today?
You:  add 1 + 2
🤖 Derma-ai: The answer to 1 + 2 is 3. How can I help you with your skin concerns today?
You:  now add 4 to it
🤖 Derma-ai: You want to add 4 to the previous result. So, 3 + 4 = 7. How can I help you with your skin concerns today?
You:  what is acne
🤖 Derma-ai: Acne, also known as Acne vulgaris, is a common skin condition that occurs when the oil-producing glands in the skin are clogged and become inflamed. It can cause a range of symptoms, including pimples, blackheads, and whiteheads, and can appear on the face, neck, chest, and back. 

According to the provided medical knowledge, Acne vulgaris is an inflammatory skin condition. If you're experiencing symptoms of acne, it's essential to practice good skin care and consider consulting a dermatologist for proper guidance and treatment. 

Would you like to know more about the causes, risks, or skin care advice related to acne?
You:  exit
